In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
from tqdm.auto import tqdm

In [3]:
df = pd.read_csv(r"/Users/yotamhadari/Desktop/SOLIO/Group_1_SOLIO2407.csv", index_col=[0,1])
group_df = pd.read_csv(r"/Users/yotamhadari/Desktop/SOLIO/Device_type_group_1.csv", index_col=[0])

In [4]:
df.head()
group_df.head()

,Device type
User ID,
1,real
2,sham
3,sham
4,sham
5,real


we want to calculate the difference between each participant's answers to the previous one.

In [5]:
def create_indiviudal_user_dfs(df):
    """Create a dictionary where keys are user_n and values and individual answer df for that user."""
    users_dfs = {}
    for user in df.index.get_level_values(0).drop_duplicates():  # Used later
        users_dfs[f"{user}"] = df.loc[df.index.get_level_values(0) == user]
    return users_dfs

In [32]:
users_dict = create_indiviudal_user_dfs(df)
user_1 = users_dict['3']
user_1.head(10)


def harvesting_mean_median(user_df, periods_to_check=30):
    """harvesting a participant's answers to a questionnaire for it's mean and median change in values"""
    user_mean_median = {}
    diff_per = 1
    while len(user_mean_median.keys()) < len(user_df.columns):
        user_diff = user_df.diff(periods=diff_per)
        user_desc = user_diff.describe()
        user_desc.replace(np.nan, 'NaN', inplace=True)
        diff_per += 1
        for column in (user_df.columns):
            
            if user_desc.loc['mean', column] != 'NaN':
                user_mean_median[column] = {'mean':user_desc.loc['mean', column], 'median': user_desc.loc['mean', column]}

        if diff_per > periods_to_check:

            for column in (user_df.columns):

                if column not in user_mean_median.keys():
                    user_mean_median[column] = {'mean': 'NEA', 'median': 'NEA'} # not enough answers
            break
    return user_mean_median

In [33]:
group_dict = {}
for group in group_df.iloc[:, 0].drop_duplicates():
    group_dict[group] = list(group_df.loc[group_df.iloc[:,0] == group].index)

group_dict

{'real': [1, 5, 7, 9, 16, 17, 21, 13, 14], 'sham': [2, 3, 4, 18, 20]}

In [35]:
users_dfs = 
for user in users_dict.keys():
    

20
7
1
9
16
21
5
2
18
14
13
17
3
4
